<a href="https://colab.research.google.com/github/OTiaggo/API-REST-em-Go/blob/main/projeto_final_ESI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação do dataset

In [1]:
import pandas as pd

# Lê o arquivo direto do GitHub
url = "https://raw.githubusercontent.com/Davyeeh/Trabalho-final-de-Engenharia-de-Sistemas-Inteligente/refs/heads/main/datset/SaoPaulo_OnlyAppartments_2024-11-25.csv"
df = pd.read_csv(url)

print("=== COLUNAS ===")
print(df.columns.tolist())

print("\n=== PRIMEIRAS LINHAS ===")
print(df.head())

print("\n=== ÚLTIMAS LINHAS ===")
print(df.tail())

=== COLUNAS ===
['ID', 'created_date', 'Price', 'below_price', 'Area', 'Adress', 'Bedrooms', 'Bathrooms', 'Parking_Spaces', 'extract_date', 'Latitude', 'Longitude']

=== PRIMEIRAS LINHAS ===
           ID          created_date    Price  below_price  Area  \
0  2621660830  2021-02-08T20:20:19Z   539000        False    43   
1  2751807421  2018-03-27T15:40:41Z  1649980        False    45   
2  2735056019  2020-07-22T21:05:19Z  1149000        False    57   
3  2758506978  2024-11-12T13:41:55Z   255000        False    55   
4  2556833428  2021-10-29T00:18:35Z   232800        False    35   

                                              Adress  Bedrooms  Bathrooms  \
0  Rua Coronel Luís Barroso 582, Santo Amaro - Sã...         1          1   
1  Rua Diogo Jácome 955, Vila Nova Conceição - Sã...         1          1   
2      Rua Fernão Dias 323, Pinheiros - São Paulo/SP         1          1   
3   Avenida Paes de Barros 411, Mooca - São Paulo/SP         1          1   
4  Rua Professor Artu

# 🧹Tratamento dos dados

## Valores nulos

In [2]:
print("=== Linhas com valores ausentes ===")
rows_with_nan = df[df.isnull().any(axis=1)]

if not rows_with_nan.empty:
    print(rows_with_nan)
else:
    print("Nenhuma linha com valores ausentes encontrada.")

=== Linhas com valores ausentes ===
               ID          created_date    Price  below_price  Area  \
1      2751807421  2018-03-27T15:40:41Z  1649980        False    45   
4      2556833428  2021-10-29T00:18:35Z   232800        False    35   
5      2757605322  2021-02-08T20:20:19Z   630000        False    35   
13     2720202635  2018-03-27T16:57:44Z   389000        False    58   
17     2622651962  2022-07-20T13:25:42Z   340000        False    45   
...           ...                   ...      ...          ...   ...   
24455  2653301727  2018-03-28T12:23:47Z  1900000        False   160   
24460  2721518746  2018-03-27T14:30:27Z  1150000        False   105   
24461  2758735720  2018-03-30T01:27:40Z  1099970        False    88   
24463  2751419295  2018-03-27T20:10:35Z  1000000        False   132   
24467  2702195937  2018-03-27T15:18:52Z   780000        False    72   

                                                  Adress  Bedrooms  Bathrooms  \
1      Rua Diogo Jácome 955, V

In [3]:
print(f"Total de linhas antes: {len(df)}")
# Remover linhas onde Latitude OU Longitude estão vazias
df = df.dropna(subset=['Latitude', 'Longitude'])

print(f"Total de linhas depois de limpar localização: {len(df)}")


Total de linhas antes: 24468
Total de linhas depois de limpar localização: 16524


## Ajustando o preço dos imóveis

### Alguns imóveis apresentam valores iguais a 0, então escolhemos fazer a média para substuir esse valores ao invés de apenas removelos. Utilizamos o IQR para ignorar outliers e ter uma média mais real

In [4]:
# Calcular a média do preço (usando apenas preços maiores que zero)
media_preco = df.loc[df['Price'] > 0, 'Price'].mean()

# Substituir os preços que são zero pela média calculada
precos_validos = df.loc[df['Price'] > 0, 'Price']

# Calcula limites para filtrar os milionários
Q1 = precos_validos.quantile(0.25)
Q3 = precos_validos.quantile(0.75)
IQR = Q3 - Q1
limite_superior = Q3 + 1.5 * IQR

# Calcula a média apenas com os preços "normais"
media_justa = precos_validos[precos_validos <= limite_superior].mean()

print(f"Média Antiga: R$ {precos_validos.mean():,.2f}")
print(f"Média Ajustada (Sem Outliers): R$ {media_justa:,.2f}")

# Substitui os zeros por essa média mais realista
df.loc[df['Price'] == 0, 'Price'] = media_justa

Média Antiga: R$ 2,115,198.88
Média Ajustada (Sem Outliers): R$ 1,602,317.30


/tmp/ipython-input-2238755218.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1602317.3015935991' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['Price'] == 0, 'Price'] = media_justa


## Remoção de linhas duplicadas

In [5]:
# Remoção de duplicatas
print(f"Linhas antes de remover duplicatas: {len(df)}")
df = df.drop_duplicates()
print(f"Linhas após remover duplicatas: {len(df)}")

Linhas antes de remover duplicatas: 16524
Linhas após remover duplicatas: 15722


## Ajuste no endereço

In [6]:
# Função que quebra o endereço em 3 partes
def separar_endereco(texto):
    # Separa a Cidade/Estado (tudo depois do " - ")
    try:
        parte1, cidade = texto.split(" - ", 1)
    except ValueError:
        parte1 = texto
        cidade = "São Paulo/SP" # Padrão caso falte

    # Separa Rua e Bairro (tudo depois da última vírgula é o bairro)
    if "," in parte1:
        # rsplit faz a quebra da direita para a esquerda (pega a última vírgula)
        rua, bairro = parte1.rsplit(",", 1)
    else:
        # Se não tem vírgula, assumimos que só tem o bairro informado
        rua = "Não Informado"
        bairro = parte1

    return pd.Series([rua.strip(), bairro.strip(), cidade.strip()])

# Aplica a função e cria as 3 novas colunas
df[['Rua', 'Bairro', 'Cidade']] = df['Adress'].apply(separar_endereco)

# Remove a coluna Adress pois não faz mais sentido deixa-la e a coluna Cidade, já
# que todos os imóveis sao de São Paulo/SP, a variância é 0
df = df.drop(columns=['Adress', 'Cidade'])

print(df[['Rua', 'Bairro',]].head())

                              Rua        Bairro
0    Rua Coronel Luís Barroso 582   Santo Amaro
2             Rua Fernão Dias 323     Pinheiros
3      Avenida Paes de Barros 411         Mooca
6       Rua Antônio Pais de Sande  Vila Mirante
7  Rua Doutor Jesuíno Maciel 2105    Campo Belo


## Remoção de outras colunas 'inúteis'

In [7]:
colunas_remover = ['ID', 'below_price']
df = df.drop(columns=colunas_remover, errors='ignore')

In [8]:
# Salvar o arquivo limpo
df.to_csv("imoveis_tratados.csv", index=False)
print(df)
print("Arquivo 'imoveis_tratados.csv' salvo com sucesso!")

               created_date      Price  Area  Bedrooms  Bathrooms  \
0      2021-02-08T20:20:19Z   539000.0    43         1          1   
2      2020-07-22T21:05:19Z  1149000.0    57         1          1   
3      2024-11-12T13:41:55Z   255000.0    55         1          1   
6      2018-03-27T19:51:51Z   210000.0    42         1          1   
7      2018-03-30T02:50:40Z   640000.0    47         1          1   
...                     ...        ...   ...       ...        ...   
24459  2024-11-05T18:09:49Z  1070000.0    92         2          3   
24462  2018-03-27T16:01:19Z  2120000.0    95         2          3   
24464  2019-02-12T16:32:26Z  1195000.0    92         2          3   
24465  2024-02-27T17:29:22Z  1970000.0   241         2          3   
24466  2018-03-27T16:17:11Z  1600000.0   100         2          3   

       Parking_Spaces extract_date   Latitude  Longitude  \
0                   1   2024-11-25 -23.647002 -46.704905   
2                   1   2024-11-25 -23.564136 -46.6